In [1]:
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.engine import *
from ignite.utils import *

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import data.read_samples as rs
import torch.optim as optim
import torch.utils.data
import torch.nn as nn

from collections import OrderedDict

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import torch
import time
import os
import gc

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 107
EPOCH = 400
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = 143
HIDDEN_UNITS = [180, 200, 250, 80, 100, 120]
K_FOLD = 1
MAT_PATH = "C:/Users/HILAB_Labtop_02/Desktop/insung/ecg-dbn/data/mit.mat"

[INFO] Pre-processing for make clean
2022-05-22 19:06:42.909555 model.py code start


In [2]:
device = torch.device('cuda')
print(torch.cuda.get_device_name(device))
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

NVIDIA GeForce RTX 3080 Laptop GPU


## RBM set-up as object

In [3]:
class RBM(nn.Module): 
    with torch.cuda.device(0):
        def __init__(self, n_vis, n_hid, k, batch):
            super(RBM, self).__init__()
            self.W      = nn.Parameter(torch.randn(1, batch, device=device) * 1e-2)
            self.n_vis  = n_vis
            self.n_hid  = n_hid
            self.k      = k
            self.batch  = batch
            self.v_bias = nn.Parameter(torch.zeros(n_vis, 1, device=device))
            self.h_bias = nn.Parameter(torch.zeros(n_hid, 1, device=device))
        
        def sample_from_p(self, p):
            return F.relu(
                torch.sign(
                    p - Variable(torch.randn(p.size(), device=device))
                )
            ).to(device=device)

        ''' ISSUE PART '''
        def v_to_h(self, v):
            w = (self.W.clone())

            p_h = F.sigmoid(
                F.linear(v, w)
            ).to(device=device)

            sample_h = self.sample_from_p(p_h)
            return p_h, sample_h

        def h_to_v(self, h):
            w = self.W.t().clone()

            p_v = F.sigmoid(
                F.linear(h, w)
            ).to(device=device)

            sample_v = self.sample_from_p(p_v)
            return p_v, sample_v
        
        def forward(self, v):
            pre_h1, h1 = self.v_to_h(v)
            h_ = h1

            for _ in range(self.k):
                pre_v_, v_ = self.h_to_v(h_)
                pre_h_, h_ = self.v_to_h(v_)
            return v, v_
        
        def get_weight(self):
            return self.W

In [4]:
class SVM(nn.Module):
    with torch.cuda.device(0):
        def __init__(self, lr, n_x):
            super(SVM, self).__init__()
            self.lr = lr
            self.fully = nn.Linear(n_x, 1).to(device=device)
        
        def forward(self, x):
            fwd = self.fully(x)
            return fwd

## Init of custom functions

In [5]:
def eval_step(engine, batch):
    return batch

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

default_evaluator = Engine(eval_step)

def get_acc(y_true, y_pred):
    metric = Accuracy()
    metric.attach(default_evaluator, "accuracy")
    state = default_evaluator.run([[y_pred, y_true]])
    return state.metrics["accuracy"]

In [6]:
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

## Pre-processing

In [7]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 122
[IWIP]		rdsamp Converting 124
[IWIP]		rdsamp Converting 201
[IWIP]		rdsamp Converting 203
[IWIP]		rdsamp Converting 205
[IWIP]		rdsamp Converting 207
[IWIP]		rdsamp Converting 208
[IWIP]		rdsamp Converting 209
[IWIP]		rdsamp Converting 215
[IWIP]		rdsamp Converting 220
[IWIP]		rdsamp Converting 223
[IWIP]		rdsamp Converting 230
[INFO] Read records file from  ./data/db2/
[RSLT]			 Export records ...
[INFO]./rdsamp commending start
[IWIP]

KeyboardInterrupt: 

In [ ]:
print(len(db1_sig[0]))

In [ ]:
print(db1_sig)

In [ ]:
train_dataloader = DataLoader(db1_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True)

test_dataloader = DataLoader(db3_sig,
                             batch_size=BATCH_SIZE,
                             num_workers=0,
                             shuffle=True)

## Call models, probabilties.

In [ ]:
bbrbm_first = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_second = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_third = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

gbrbm_first = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[3], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_second = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[4], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_third = RBM(n_vis=VISIBLE_UNITS, n_hid=HIDDEN_UNITS[5], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

first_train_op = optim.Adagrad(bbrbm_first.parameters(), LEARNING_RATE)
second_train_op = optim.Adagrad(bbrbm_second.parameters(), LEARNING_RATE)
third_train_op = optim.Adagrad(bbrbm_third.parameters(), LEARNING_RATE)

gb_first_train_op = optim.Adagrad(gbrbm_first.parameters(), LEARNING_RATE)
gb_second_train_op = optim.Adagrad(gbrbm_second.parameters(), LEARNING_RATE)
gb_third_train_op = optim.Adagrad(gbrbm_third.parameters(), LEARNING_RATE)

omse_loss = list()
output_gb = list()
best_acc = float()
svm_best_acc = float()
mse_loss = nn.MSELoss()

# gaussian_std = torch.arange(1, 0, -0.00537, device=device)
gaussian_std = torch.arange(1, 0, -0.0094, device=device)
print(gaussian_std.size())

svm_model = SVM(lr=LEARNING_RATE, n_x=107)
svm_optimizer = optim.Adagrad(svm_model.parameters(), lr=LEARNING_RATE)

## Train Model
Always careful at this part. Some of setting are can be occure system errors like cuda stuff.

In [ ]:
'''BBRBM Train Part'''

loss_ = []
output_bb = []
model_path_str = str()

print("RBM START!")

for epoch in range(EPOCH):
    tmp_acc = float()
    run_acc = float()
    start = time.time()
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        if i == 10051:
            break

        data = Variable(torch.tensor(data.uniform_(0, 1), dtype=torch.float32))
        
        sample_data = torch.bernoulli(data).view(-1, BATCH_SIZE).to(device=device)
        fs_data = sample_data
        
        # tensor binary
        fvog_first, v1 = bbrbm_first(sample_data)
        omse_loss = mse_loss(fvog_first, v1)
        
        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
    
    for _, (data) in enumerate(v1): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).to(device=device)

        # tensor binary
        vog_second, v2 = bbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()
    
    for _, (data) in enumerate(v2):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).to(device=device)

        vog_third, v3 = bbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()

        run_acc += (sample_data == v3).sum().item()
    
    '''
        GBRBM GBRBM GBRBM GBRBM GBRBM GBRBM GBRBM 
    '''

    for i, (data) in enumerate(output_bb):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        # tensor binary
        vog_first, v1 = gbrbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        gb_first_train_op.zero_grad()
        gb_first_train_op.step()
        omse_loss.backward()

    for _, (data) in enumerate(v1): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        # tensor binary
        vog_second, v2 = gbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        gb_second_train_op.zero_grad()
        omse_loss.backward()
        gb_second_train_op.step()

    for _, (data) in enumerate(v2):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        vog_third, v3_e = gbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3_e)
        
        gb_third_train_op.zero_grad()
        omse_loss.backward()
        gb_third_train_op.step()

    svm_X = torch.tensor(v3_e, dtype=torch.float32, device=device)
    svm_Y = torch.tensor(Y, dtype=torch.float32, device=device)
    N = len(svm_Y)

    perm = torch.randperm(N, device=device)

    for i in range(0, N, BATCH_SIZE):
        correct = float()

        x = torch.tensor(svm_X.clone().detach(), device=device)
        y = torch.tensor(svm_Y.clone().detach(), device=device)

        # Forward
        output = svm_model(x)
        
        # Backward
        svm_optimizer.zero_grad()        
        svm_optimizer.step()

        predicted = torch.tensor(output.data >= 0, dtype=torch.float32)

        svm_acc = output.data >= predicted
        
    svm_best_acc = svm_acc
    svm_path = "./mat_svm_model/" + str(epoch) + "_Train_svm_model_acc__.pth"
    torch.save(svm_model.state_dict(), svm_path)

    acc_v = (vog_third >= 0).float()
    acc = get_acc(
        acc_v, v3_e
    ) * 100
    
    if acc > best_acc:
        best_acc = acc    
        
        path = "./say_cheese/ahh_saveMode_through_" + str(epoch) + "_" + str(acc) + "GBRBM.pth"
        model_path_str = path
        torch.save(gbrbm_third.state_dict(), path)
    output_gb.append(v3_e)

    print("GB-DBN Training loss for {0}th epoch {1}\tEstimate time : {2}\tAcc : {3}\t\tBest Acc : {4}\tSVM Acc & Predicted: {5}, {6}".format(epoch + 1, omse_loss, time.time() - start, acc, best_acc, svm_acc, predicted))
    gc.collect()

In [ ]:
plt.plot(output_gb, len(output_gb))

In [ ]:
LOAD_PATH = model_path_str

load_model = RBM(n_vis=187, n_hid=120, k=K_FOLD, batch=BATCH_SIZE)
load_model.load_state_dict((torch.load(LOAD_PATH)))
load_model.to(device=device)

for i, (data) in enumerate(test_dataloader):
    if i == 939:
        continue

    data = Variable(
            torch.tensor(data, dtype=torch.float32)
    ).uniform_(0, 1)
    
    sample_data = torch.bernoulli(data).view(-1, 107).to(device=device)
    
    # tensor binary
    vog_first, v1 = load_model(sample_data)
    omse_loss = mse_loss(vog_first, v1)
    
    first_train_op.zero_grad()
    first_train_op.step()
    omse_loss.backward()

print("Load model: ", LOAD_PATH)
print("Acc : ", get_acc(vog_first, v1))

In [ ]:
SVM_LOAD_PATH = "./mat_svm_model/7_Train_svm_model_acc__.pth"

svm_model_load = SVM(lr=LEARNING_RATE, n_x=107)
svm_model_load.load_state_dict(torch.load(SVM_LOAD_PATH))
svm_model_load.to(device=device)

best_svm_acc = 0.

for i in range(0, N, BATCH_SIZE):
        correct = float()

        x = torch.tensor(svm_X.clone().detach(), device=device)
        y = torch.tensor(svm_Y.clone().detach(), device=device)

        # Forward
        output = svm_model(x)
        
        # Backward
        svm_optimizer.zero_grad()        
        svm_optimizer.step()

        predicted = torch.tensor(output.data >= 0, dtype=torch.float32)
        svm_acc = output.data >= predicted

        if svm_acc >= best_svm_acc:
                best_svm_acc = svm_acc

print("SVM Model Predicted: ", predicted, "Accuracy: ", best_svm_acc)